# Choose chunk size

```python
faithfulness_new_prompt_template = """ Hãy cho biết liệu một thông tin nhất định có được hỗ trợ trực tiếp bởi ngữ cảnh hay không.
    Bạn cần trả lời bằng YES hoặc NO.
    Trả lời YES nếu bất kỳ phần nào của ngữ cảnh hỗ trợ rõ ràng thông tin, ngay cả khi phần lớn ngữ cảnh không liên quan. Nếu ngữ cảnh không hỗ trợ rõ ràng thông tin, hãy trả lời NO. Một số ví dụ được cung cấp bên dưới.

    Thông tin: Bánh táo thường có hai lớp vỏ.
    Ngữ cảnh: Bánh táo là một loại bánh trái cây trong đó thành phần nhân chính là táo.
    Bánh táo thường được phục vụ với kem tươi đánh bông, kem ('bánh táo à la mode'), trứng sữa hoặc phô mai cheddar.
    Nó thường có hai lớp vỏ, với lớp vỏ bánh ngọt cả trên và dưới nhân; lớp vỏ trên có thể đặc hoặc dạng lưới (đan xen các dải ngang).
    Trả lời: YES

    Thông tin: Bánh táo có vị tệ.
    Ngữ cảnh: Bánh táo là một loại bánh trái cây trong đó thành phần nhân chính là táo.
    Bánh táo thường được phục vụ với kem tươi đánh bông, kem ('bánh táo à la mode'), trứng sữa hoặc phô mai cheddar.
    Nó thường có hai lớp vỏ, với lớp vỏ bánh ngọt cả trên và dưới nhân; lớp vỏ trên có thể đặc hoặc dạng lưới (đan xen các dải ngang).
    Trả lời: NO

    Thông tin: Paris là thủ đô của Pháp.
    Ngữ cảnh: Tài liệu này mô tả một chuyến đi trong ngày ở Paris. Bạn sẽ ghé thăm các địa danh nổi tiếng như Tháp Eiffel, Bảo tàng Louvre và Nhà thờ Đức Bà.
    Trả lời: NO

    Thông tin: {query_str}
    Ngữ cảnh: {context_str}
    Trả lời:

    """

```

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama
from pydantic import BaseModel, Field

llm_model_name='qwen2.5:7b'
llm = ChatOllama(model=llm_model_name, temperature=0)

class GradeChunk(BaseModel):
    """Binary score for relevance check on chunks."""

    binary_score: str = Field(
        description="Piece of information is directly supported by the context, 'yes' or 'no'"
    )

structured_llm_grader = llm.with_structured_output(GradeChunk)

faithfulness_new_prompt_template = """ Please tell if a given piece of information is directly supported by the context.
    You need to answer with either YES or NO.
    Answer YES if any part of the context explicitly supports the information, even if most of the context is unrelated. If the context does not explicitly support the information, answer NO. Some examples are provided below.

    Information: Apple pie is generally double-crusted.
    Context: An apple pie is a fruit pie in which the principal filling ingredient is apples.
    Apple pie is often served with whipped cream, ice cream ('apple pie à la mode'), custard, or cheddar cheese.
    It is generally double-crusted, with pastry both above and below the filling; the upper crust may be solid or latticed (woven of crosswise strips).
    Answer: YES

    Information: Apple pies taste bad.
    Context: An apple pie is a fruit pie in which the principal filling ingredient is apples.
    Apple pie is often served with whipped cream, ice cream ('apple pie à la mode'), custard, or cheddar cheese.
    It is generally double-crusted, with pastry both above and below the filling; the upper crust may be solid or latticed (woven of crosswise strips).
    Answer: NO

    Information: Paris is the capital of France.
    Context: This document describes a day trip in Paris. You will visit famous landmarks like the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral.
    Answer: NO

    Information: {query_str}
    Context: {context_str}
    Answer:

    """
    
prompt = PromptTemplate(
    template=faithfulness_new_prompt_template, 
    input_variables=["query_str", "context_str"])

chunk_grader = prompt | structured_llm_grader
    


### Function to evaluate metrics for each chunk size